In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
from concurrent.futures import ThreadPoolExecutor

In [57]:
url = f"https://go.drugbank.com/bio_entities/BE0000180"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
entity_name = soup.find("h1").text
main_tag = soup.find('main')
syn_link = main_tag.find('a', href=lambda href: href and href.startswith('/'))
link = f"https://go.drugbank.com{syn_link['href']}"

response = requests.get(link)
soup = BeautifulSoup(response.content, "html.parser")

synonyms_section = soup.find('dt', {'id': 'synonyms'})
synonyms_list = synonyms_section.find_next('ul')
syns = [synonym.text for synonym in synonyms_list if synonym.text != '\n']

In [8]:
def get_entity_name(id):
    url = f"https://go.drugbank.com/bio_entities/{id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    entity_name = soup.find("h1").text
    main_tag = soup.find('main')
    if (entity_name == "This page doesn't exist. What a pain."):
        syns = 0
    else:
        try:
            syn_link = main_tag.find('a', href=lambda href: href and href.startswith('/'))
        
            link = f"https://go.drugbank.com{syn_link['href']}"

            response = requests.get(link)
            soup = BeautifulSoup(response.content, "html.parser")
            try:
                synonyms_section = soup.find('dt', {'id': 'synonyms'})
                synonyms_list = synonyms_section.find_next('ul')
                syns = [synonym.text for synonym in synonyms_list if synonym.text != '\n']
            except:
                syns = 0
        except:
            syns = 0

    return entity_name, syns

In [ ]:
# get target names only
# id :BE0000001~BE0010215
# parallel crawler
num_targets = 10215
num_workers = 3
freq = 0.8

target_vocabulary = {}

for n in range(1, num_targets+1, num_workers):

    ids = [str(f'BE00{(5-len(str(i)))*"0"}{i}') for i in range(n, n+num_workers)]
    if n + num_workers > num_targets:
        ids = [str(f'BE00{(5-len(str(i)))*"0"}{i}') for i in range(n, num_targets+1)]

    with ThreadPoolExecutor(max_workers=num_workers ) as executor:
        results = executor.map(get_entity_name, ids)
        # for result in results:
        #     print(result)

    for id, entity_name in zip(ids, results):
        target_vocabulary[id] = entity_name
    # print(entity_name)

    if (n - 1) % 99 == 0:
        with open('./DBid_to_names/target vocabulary.json', 'w') as f:
            f.write(json.dumps(target_vocabulary))

    time.sleep(freq)

with open('./DBid_to_names/target vocabulary.json', 'w') as f:
    f.write(json.dumps(target_vocabulary))

In [11]:
# get syn
# id :BE0000001~BE0010215
# parallel crawler
num_targets = 10215
num_workers = 3
freq = 0.5

# target_vocabulary = {}
# target_syn_vocabulary = {}

for n in range(7235, num_targets+1, num_workers):

    ids = [str(f'BE00{(5-len(str(i)))*"0"}{i}') for i in range(n, n+num_workers)]
    if n + num_workers > num_targets:
        ids = [str(f'BE00{(5-len(str(i)))*"0"}{i}') for i in range(n, num_targets+1)]

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(get_entity_name, ids)
        # for result in results:
        #     print(result)

    for id, (entity_name, syn_names) in zip(ids, results):
        # target_vocabulary[id] = entity_name
        if entity_name != "This page doesn't exist. What a pain.":
            target_syn_vocabulary[entity_name] = syn_names
    # print(entity_name)

    if (n - 1) % 99 == 0:
        # retain 2 decimals while print out the progress n / num_targets
        print(f'{((n + 1) / num_targets):.2%}')
        with open('/home/tian/Projects/MyReaserch/DBid_to_names/target vocabulary w syn.json', 'w') as f:
            f.write(json.dumps(target_syn_vocabulary))

    time.sleep(freq)

with open('/home/tian/Projects/MyReaserch/DBid_to_names/target vocabulary w syn.json', 'w') as f:
    f.write(json.dumps(target_syn_vocabulary))

In [12]:
n

10214

In [13]:
len(target_syn_vocabulary)

5205

In [14]:
with open('/home/tian/Projects/MyReaserch/DBid_to_names/target vocabulary w syn.json', 'w') as f:
    f.write(json.dumps(target_syn_vocabulary, indent=4, separators=(", ", ": ")))

In [53]:
len(target_vocabulary)

10215